La extracción y consulta es del Banco Mundial contiene alrededor de 30 indicadores educativos internacionalmente comparables para el acceso, la progresión, la finalización, la alfabetización, los docentes, la población y los gastos.

Los indicadores cubren el ciclo educativo desde la educación preescolar hasta la educación terciaria.

https://databank.worldbank.org/source/education-statistics-%5E-all-indicators/Type/TABLE/preview/on



In [22]:
import pandas as pd
import numpy as np
import pycountry

                                Reporte de calidad de datos & normalizacion

In [51]:
data_educacion = pd.read_csv('indicador_educacion.csv')

In [52]:
print("Número de filas:", data_educacion.shape[0])
print("Número de columnas:", data_educacion.shape[1])

Número de filas: 6510
Número de columnas: 11


In [53]:
data_educacion.head() #visualizamos la data

,Country Name,Country Code,Series,Series Code,1990,1995,2000,2005,2010,2015,2020
0,Afghanistan,AFG,Government expenditure on education as % of GD...,SE.XPD.TOTL.GD.ZS,..,..,..,..,3.47945,3.2558,..
1,Afghanistan,AFG,"Out-of-school children of primary school age, ...",SE.PRM.UNER,..,..,..,..,..,..,..
2,Afghanistan,AFG,Proportion of teachers with the minimum requir...,SE.PRM.TCAQ.ZS,..,..,..,..,..,..,..
3,Afghanistan,AFG,Proportion of teachers with the minimum requir...,UIS.TRTP.02,..,..,..,..,..,..,..
4,Afghanistan,AFG,Proportion of teachers with the minimum requir...,UIS.TRTP.2,..,..,..,..,..,..,..


Se dropeara la columna Series code no aporta mucho al modelo

In [54]:
data_educacion.drop(columns={'Series Code'},inplace=True)

In [55]:
data_educacion.isna().sum() #vemos si hay valores vacios

Country Name    0
Country Code    0
Series          0
1990            0
1995            0
2000            0
2005            0
2010            0
2015            0
2020            0
dtype: int64

In [56]:
data_educacion.isnull().sum() ##verificamos si existen valores nulos

Country Name    0
Country Code    0
Series          0
1990            0
1995            0
2000            0
2005            0
2010            0
2015            0
2020            0
dtype: int64

In [57]:
j = []
for i in data_educacion.iloc[:,4:10].columns:
    j.append(len(data_educacion[data_educacion[i] == '..']))
print('Numero total de valores faltantes: ' + str(np.sum(j) ))

Numero total de valores faltantes: 26895


In [58]:
data_educacion.shape

(6510, 10)

In [59]:
print("numero total de valores:" + str(6540 * 10))

numero total de valores:65400


In [60]:
print("porcentaje de valores faltantes: " + str((26895/65400) * 100) + "%")

porcentaje de valores faltantes: 41.12385321100918%


In [61]:
#Normalizacion nombre de pais
def normalize_country(name):

    custom_mapping = {
        "Iran (Islamic Republic of)": "iran",
        "côte d'ivoire": "cote d'ivoire",
        "China, Hong Kong SAR*": "hong kong",
        "hong kong sar, china": "hong kong",
        "iran, islamic rep.": "iran",
        "spain*": "spain",
        "turkey":"turkiye"
    }
    
    if name in custom_mapping:
        return custom_mapping[name]
    try:
        country = pycountry.countries.lookup(name)
        return country.name
    except LookupError:
        if name == "United States of America*":
            return "United States"
        elif name == "Iran (Islamic Republic of)":
            return "iran"
        elif name == "Russian Federation":
            return "Russia"
        elif name == "France*":
            return "France"
        elif name == "Australia*":
            return "Australia"
        elif name == "United Kingdom*":
            return "United Kingdom"
        elif name == "China, Hong Kong SAR*":
            return "hong kong"                 
        elif name == "Ukraine*":
            return "Ukraine"
        # agrega más verificaciones aquí para otros casos especiales
        else:
          return name

In [62]:
data_educacion['Country Name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Faroe Is

In [63]:
data_educacion['Country Name'] = data_educacion['Country Name'].str.strip()

In [64]:
data_educacion['Country Name'] = data_educacion['Country Name'].replace({"Côte d'Ivoire" : "cote d'ivore"})

In [65]:
data_educacion['Country Name'] = data_educacion['Country Name'].apply(normalize_country).str.lower()

In [66]:
#creamos un codigo por pais
y = 0
data_educacion['cod'] = 0
l = []
for i in range(len(data_educacion)):
  if data_educacion['Country Code'].iloc[i] not in l:
    y += 1
    l.append(data_educacion['Country Code'].iloc[i])
    data_educacion['cod'].iloc[i] = y
  else:
    data_educacion['cod'].iloc[i] = y

C:\Users\USER\AppData\Local\Temp\ipykernel_10468\2953554771.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_educacion['cod'].iloc[i] = y
C:\Users\USER\AppData\Local\Temp\ipykernel_10468\2953554771.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_educacion['cod'].iloc[i] = y


In [67]:
len(data_educacion.Series.unique())

30

Resumén de análisis de la data

        *No hay valores nulos

        *No existen duplicados 

        *EL dataset tiene en total 30 Series clave diferentes en el sector educación de cada pais.
        
        *En la columna años existen valores faltantes declarados asi (..) dicho de otra forma:

        --No existen datos en tal año para ese país con la descripción de la serie sobre la que este el indice.
        
        

In [68]:
data_educacion.to_csv('indicador_educacion_normalizado.csv',index=False)